In [24]:
## Carga de librerias
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
warnings.filterwarnings('ignore')

##Forma
from IPython.display import display, Markdown
palette_color=['#d4afb9', '#d1cfe2', '#9cadce', '#7ec4cf', '#52b2cf']

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
path='/content/drive/MyDrive/AA/Analitica-en-Marketing'
os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

In [28]:
import funciones as fn ### importar modulo de  funciones
from funciones import * ### importar toddas llas funciones del archivo

## con esto ya ses pueden utilizar todas las funciones propia

In [29]:
##### conectarse a BD #######
conn = sql.connect('/content/drive/MyDrive/AA/Analitica-en-Marketing/Data/db_movies2')
cur=conn.cursor()

In [30]:
### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',)]

In [31]:
## traer tabla de BD a python ####
movies= pd.read_sql("select *  from movies", conn)
ratings = pd.read_sql('select * from ratings', conn)

In [32]:
def check_df(dataframe):
    # Dimensiones base general
    display(Markdown('**Dimensiones base general**'))
    display(dataframe.shape)

    # Dimensiones sin duplicados
    display(Markdown('**Dimensiones sin duplicados**'))
    display(dataframe.drop_duplicates().shape)

    # Tipos de datos
    display(Markdown('**Tipos**'))
    display(dataframe.dtypes)

    # Valores nulos
    display(Markdown('**Nulos**'))
    display(dataframe.isnull().sum())

check_df(movies)
check_df(ratings)

**Dimensiones base general**

(9742, 3)

**Dimensiones sin duplicados**

(9742, 3)

**Tipos**

...

(100836, 4)

**Tipos**

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


**Nulos**

,0
userId,0
movieId,0
rating,0
timestamp,0


In [ ]:
### ver el tipo de datos y faltantes
print(movies.info())
print(ratings.info())

In [ ]:
# distribución de las calificaciones
cr=pd.read_sql(""" select rating,
                    count(*) as conteo
                    from ratings
                    group by rating
                    order by conteo desc""", conn)
cr

In [ ]:
data  = go.Bar( x=cr.rating,marker_color=palette_color,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)

In [ ]:
# Cacular cada usuario cuantas peliculas calificó
rating_users = pd.read_sql(''' select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                order by cnt_rat asc
                                ''',conn )

In [ ]:
fig  = px.histogram(rating_users, x= 'cnt_rat', color_discrete_sequence=palette_color, title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()


In [ ]:
## Descripción ratings
rating_users.describe()

In [ ]:
## excluir usuarios con menos de 500 libros calificados
rating_users2 = pd.read_sql('''select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                having cnt_rat <=500
                                order by cnt_rat asc
                                ''',conn )
rating_users2

In [ ]:
rating_users2.describe()

In [ ]:
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [ ]:
### calificacion de cada pelicula
rating_movie = pd.read_sql('''select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                order by cnt_rat desc
                                ''',conn )

In [ ]:
fig  = px.histogram(rating_movie, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [ ]:
rating_movie.describe()

In [ ]:
####peliculas que tengan más de 10 calificaciones
rating_movie2=pd.read_sql(''' select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                having cnt_rat >= 10
                                order by cnt_rat desc
                                ''',conn )

In [ ]:
fig  = px.histogram(rating_movie2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [ ]:
rating_movie2.describe()

In [35]:
sql_file_path = '/content/drive/MyDrive/AA/Analitica-en-Marketing/Preprocesamiento.sql'

In [36]:
# Leer el archivo SQL desde Google Drive
with open(sql_file_path, 'r') as sql_file:
    sql_script = sql_file.read()

# Ejecutar la función fn.ejecutar_sql con el contenido del archivo SQL
fn.ejecutar_sql(sql_script, cur)

OSError: [Errno 36] File name too long: "SELECT * FROM movies;\n-- Añadir una nueva columna para el título limpio\nALTER TABLE movies\nADD COLUMN clean_title VARCHAR(255);\n\n-- Añadir una nueva columna para el año\nALTER TABLE movies\nADD COLUMN year INT;\n\n-- Actualizar la tabla para llenar las nuevas columnas con los datos limpios\nUPDATE movies\nSET year = CAST(SUBSTRING(title, LENGTH(title) - 4, 4) AS INT), -- Extraer el año\n    clean_title = TRIM(SUBSTRING(title, 1, LENGTH(title) - 6)) -- Extraer el título limpio\nWHERE CHARINDEX('(', title) > 0 AND CHARINDEX(')', title, CHARINDEX('(', title)) > 0;\n\n-- Visualizar el resultado\n\nSELECT * FROM movies;\n\n--- ver tabla ratings\n\nSELECT * FROM ratings;\n\n-- Crear tabla con usuarios que han calificado más de 20 y menos de 1000 películas\nDROP TABLE IF EXISTS usuarios_sel;\n\nCREATE TABLE usuarios_sel AS\nSELECT userId AS user_id, COUNT(*) AS cnt_rat\nFROM ratings\nGROUP BY userId\nHAVING cnt_rat <= 1000\nORDER BY cnt_rat DESC;\